# Google Drive Downloader For Google Colab

This notebook allows you to download a revision of a file from Google Drive to your Google Colab workspace.

Normally, when you mount your drive into your runner, file revisions are not visible on the list, therefore you cannot copy the file to your workspace directly.

You can access the revision by using downloadLink through Drive console, however, to download directly into your workspace via HTTP,  you need to be authenticated.

Also you will be out-of-memory if you try to download the files larger than your runner's memory.

The notebook allows you to see your files and revisions, goes through required authentication steps and downloads the file in chunks.

## Prerequistes

Have a look at the [references](##References) first.

1. Create a project under Google Cloud Console https://console.cloud.google.com/
2. Create OAuth client ID in https://console.cloud.google.com/apis/credentials
- Create consent screen if you don't have yet https://console.cloud.google.com/apis/credentials/consent
- Add your user email under test users
- On create client id page:
 - Application type: Web application
 - Authorised redirect URIs: http://localhost:8081/
3. Download client secrets json, upload to your workspace.

## References

- https://support.google.com/cloud/answer/6158849?hl=en
- https://developers.google.com/identity/protocols/oauth2

In [ ]:
#@title Install the requirements

!pip install -r requirements.txt

In [ ]:
#@title Get the authorization code

import requests
import webbrowser
import json

client_secret_file = "client_secret.json" #@param {type:"string"}

f = open(client_secret_file, 'r')
client_secrets = json.load(f)['web']

authorization_url = client_secrets['auth_uri']
token_url = client_secrets['token_uri']
client_id = client_secrets['client_id']
client_secret = client_secrets['client_secret']
redirect_uri = client_secrets['redirect_uris'][0]
scope = 'https://www.googleapis.com/auth/drive'
response_type = 'code'

# Construct the authorization URL
authorization_params = {
    'client_id': client_id,
    'redirect_uri': redirect_uri,
    'scope': scope,
    'response_type': response_type
}
authorization_url = f'{authorization_url}?{"&".join(f"{k}={v}" for k, v in authorization_params.items())}'

print("Visit the returned URL and copy the authorizarion code:", authorization_url)


In [ ]:
#@title Exchange authorization code for credentials

from google_auth_oauthlib.flow import Flow
import os 

os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1'

authorization_code = '4/0###' #@param {type:"string"}

# Set up the OAuth 2.0 flow for web applications
flow = Flow.from_client_secrets_file(
    client_secret_file,
    scopes=[scope],
    redirect_uri=redirect_uri
)

flow.fetch_token(code=authorization_code)
credentials = flow.credentials

In [ ]:
#@title List all files in drive

from googleapiclient.discovery import build

service = build('drive', 'v3', credentials=credentials)

response = service.files().list().execute()
files = response.get('files', [])

if files:
    for file in files:
        print(file)
else:
    print('No files found.')

In [ ]:
#@title Get revisions of a file
file_id = '###' #@param {type:"string"}

response = service.revisions().list(fileId=file_id).execute()
revisions = response.get('revisions', [])

if revisions:
    for rev in revisions:
        print(rev)
else:
    print('No revisions found.')

In [ ]:
#@title Get file size and name for a revision
revision_id = '###' #@param {type:"string"}

revision = service.revisions().get(
    fileId=file_id,
    revisionId=revision_id,
    fields='size, originalFilename'
).execute()

file_size = revision.get('size')
file_name = revision.get('originalFileName', 'untitled')

if file_size:
    print(f'File Size for {file_name}: {file_size} bytes')
else:
    print('File size not available for this revision.')

In [ ]:
#@title Exchange authorization code for an access token

#@markdown [Generate](#scrollTo=Gs4zKNTuSlml&line=1&uniqifier=1) a new authorization code.

new_authorization_code = '4/0###' #@param {type:"string"}

token_data = {
    'code': new_authorization_code,
    'client_id': client_id,
    'client_secret': client_secret,
    'redirect_uri': redirect_uri,
    'grant_type': 'authorization_code'
}

token_response = requests.post(token_url, data=token_data)
access_token = token_response.json()['access_token']
print(access_token)

In [ ]:
#@title Download the file in chunks

# Check existing file. If file exists, continue from where it left
import sys

file_root = './' #@param {type:"string"}
chunk_size = 10240 #@param {type:"integer"}

file_path = os.path.join(file_root, file_name) 
process = {}

if os.path.exists(file_path):
    process["start_byte"] = os.path.getsize(file_path)
    process["mode"] = "ab"
    process["download"] = "As resume"
else:
    process["start_byte"] = 0
    process["mode"] = "wb"
    process["download"] = "As a new file"
if process["start_byte"] == file_size:
    sys.exit("The download of this file has already been finished.")

# Download process
print(process["download"])
headers = {
    "Authorization": f"Bearer {access_token}",
    "Range": f'bytes={process["start_byte"]}-',
}
url = f"https://www.googleapis.com/drive/v2/files/{file_id}?revisionId={revision_id}&alt=media&source=downloadUrl"
with requests.get(url, headers=headers, stream=True) as r:
    r.raise_for_status()
    with open(file_path, process["mode"]) as f:
        for chunk in r.iter_content(chunk_size=chunk_size):
            f.write(chunk)